# Интерфейсы scikit-learn

Авторы: Гирдюк Дмитрий, Антон Першин

In [ ]:
from __future__ import annotations

import numpy as np

from numpy.typing import NDArray
from sklearn.base import BaseEstimator, OneToOneFeatureMixin, TransformerMixin
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

## Estimator

Для примера построим простой estimator, который в перспективе будет вычитать из признаков их среднее значение и после сдвигать признаки на заранее заданную константу.

In [ ]:
class SubtractMeanAndShiftEstimator(BaseEstimator):
    def __init__(self, shift: float = 0.) -> None:
        self.shift = shift
        # we add a trailing underscore for parameters which will be learnt in fit()
        self.means_: NDArray[np.float_] | None = None

    def fit(self, X: NDArray[np.float_], y: NDArray[np.float_] | None = None) -> SubtractMeanAndShiftEstimator:
        # y is ignored here
        self.means_ = X.mean(axis=0)  # the first axis corresponds to samples by default
        return self

In [ ]:
estimator = SubtractMeanAndShiftEstimator(shift=3.0)

Метод `get_params()` реализован в `BaseEstimator`, и мы можем сразу использовать его для получения гиперпараметров модели. Это возможно, так как единственный гиперпараметр `shift` был передан как явное ключевое слово в контрукторе

Обратите внимание, что соответствующий аттрибут класса должен совпадать с ключевым словом: `self.shift = shift`

In [ ]:
estimator.get_params()

Аналогично мы можем использовать `set_params()` для задания значений гиперпараметров. Этот метод пригодится при поиске оптимальных значений гиперпараметров

In [ ]:
estimator.set_params(shift=5.0)
estimator.get_params()

In [ ]:
X = np.array([[1, 10], [3, 30], [2, 20]])
y = np.array([[ 0, -9], [ 2, 11], [ 1,  1]])
estimator.fit(X, y)

## Predictor

Рассмотрим тот же класс, но добавим к нему методы `predict()` и `score()`

In [ ]:
class SubtractMeanAndShiftPredictor(BaseEstimator):
    def __init__(self, shift: float = 0.) -> None:
        self.shift = shift
        # we add a trailing underscore for parameters which will be learnt in fit()
        self.means_: NDArray[np.float_] | None = None

    def fit(self, X: NDArray[np.float_], y: NDArray[np.float_] | None = None) -> SubtractMeanAndShiftEstimator:
        # y is ignored here
        self.means_ = X.mean(axis=0)  # the first axis corresponds to samples by default
        return self

    def predict(self, X: NDArray[np.float_]) -> NDArray[np.float_]:
        return X - (np.ones((X.shape[0], 1)) @ self.means_.reshape(-1, 1).T) + self.shift

    def score(self, X: NDArray[np.float_], y: NDArray[np.float_]) -> float:
        # R2 \in (-\infty; 1] is the coefficient of determination
        return r2_score(y, self.predict(X))

In [ ]:
predictor = SubtractMeanAndShiftPredictor(shift=1.0)
predictor.fit(X)
predictor.predict(X)
predictor.score(X, y)

## Transformer

Рассмотрим тот же класс, но добавим к нему метод `transform()`

In [ ]:
class SubtractMeanAndShiftTransformer(BaseEstimator, TransformerMixin, OneToOneFeatureMixin):
    def __init__(self, shift: float = 0.) -> None:
        self.shift = shift
        # we add a trailing underscore for parameters which will be learnt in fit()
        self.means_: NDArray[np.float_] | None = None
        self.n_features_in_: int | None = None

    def fit(self, X: NDArray[np.float_], y: NDArray[np.float_] | None = None) -> SubtractMeanAndShiftEstimator:
        # y is ignored here
        self.means_ = X.mean(axis=0)  # the first axis corresponds to samples by default
        self.n_features_in_ = X.shape[1]  # need for 'OneToOneFeatureMixin'
        return self

    def transform(self, X: NDArray[np.float_]) -> NDArray[np.float_]:
        return X - (np.ones((X.shape[0], 1)) @ self.means_.reshape(-1, 1).T) + self.shift

In [ ]:
transformer = SubtractMeanAndShiftTransformer(shift=5.0)
transformer.fit(X)
transformer.transform(X)

Так как мы добавили `TransformerMixin`, мы можем использовать метод `fit_transform()`, не реализуя его явно

In [ ]:
transformer.fit_transform(X)

Аналогично мы можем использовать метод `get_feature_names_out()`, так как мы добавили `OneToOneFeatureMixin`

In [ ]:
transformer.get_feature_names_out(input_features=["x", "y"])

## Pipelines

С помощью Pipeline мы можем производить последовательную обработку данных и выполнять предсказание в конце

In [ ]:
X = np.array([[1, 10], [3, 30], [2, 20]])
y = np.array([[0], [2], [1]])

pipeline = Pipeline(
    [("shifter", SubtractMeanAndShiftTransformer(shift=5.0)), ("regressor", LinearRegression())]
)

pipeline.fit(X, y)
y_pred = pipeline.predict(X)
print(y_pred)

Pipeline хранит последовательные Estimators в аттрибуте `steps`

In [ ]:
pipeline.steps

Перейти к объекту i-го Estimator можно напрямую через `pipeline[i]`:

In [ ]:
pipeline[0]

In [ ]:
pipeline[1].coef_

Так как Pipeline сам является Estimator, мы можем увидеть список его параметров:

In [ ]:
pipeline.get_params()

Видно, параметры промежуточных Estimator указаны как `<estimator>__<parameter>`. Следовательно, мы можем изменить параметры любого промежуточного Estimator:

In [ ]:
pipeline.set_params(shifter__shift=10)
pipeline.get_params()